### Distributed DL training with `HorovodEstimator` API.

This notebook performs distributed fitting of a fully-connected deep neural network on MNIST data in a Spark DataFrame. The model training example is adapted from Uber's [tensorflow_mnist_estimator example script](https://github.com/uber/horovod/blob/9bdd70dad6d35fe7fa7c4b0875a27c5cf668ca83/examples/tensorflow_mnist_estimator.py).

The notebook runs without code changes on CPU or GPU-enabled Spark clusters of two or more machines, and supports multi-GPU training (training with multiple GPUs per machine).

To run the notebook, use Databricks Runtime ML.

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import learn
import horovod.tensorflow as hvd

from pyspark.sql.types import *
from pyspark.sql.functions import rand, when

from sparkdl.estimators.horovod_estimator.estimator import HorovodEstimator

Using TensorFlow backend.

Load a DataFrame containing MNIST data. In practice, the loaded DataFrame might be the output of some ETL done in Spark.

In [0]:
# Load MNIST dataset, with images represented as arrays of floats
mnist_dataset = learn.datasets.mnist.load_mnist("/tmp/sid/tfrecords")
train_images, train_labels = mnist_dataset.train.images, mnist_dataset.train.labels
data = [(train_images[i].tolist(), int(train_labels[i])) for i in range(len(train_images))]
schema = StructType([StructField("image", ArrayType(FloatType())),
                     StructField("label_col", LongType())])
df = spark.createDataFrame(data, schema)

WARNING:tensorflow:From <command-3638925182337089>:2: load_mnist (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:300: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/sid/tfrecords/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/sid/tfrecords/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/sid/tfrecords/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/sid/tfrecords/t10k-labels-idx1-ubyte.gz
WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

Print out the HorovodEstimator API signature for reference.

In [0]:
help(HorovodEstimator)

Help on class HorovodEstimator in module sparkdl.estimators.horovod_estimator.estimator:

class HorovodEstimator(pyspark.ml.base.Estimator, sparkdl.estimators.horovod_estimator.params.HasModelFunction, sparkdl.estimators.horovod_estimator.params.HasFeatureMapping, sparkdl.estimators.horovod_estimator.params.HasLabelCol, sparkdl.estimators.horovod_estimator.params.HasModelDir, sparkdl.estimators.horovod_estimator.params.HasMaxSteps, sparkdl.estimators.horovod_estimator.params.HasBatchSize, sparkdl.estimators.horovod_estimator.params.HasIsValidationCol, sparkdl.estimators.horovod_estimator.params.HasModelFunctionParams, sparkdl.estimators.horovod_estimator.params.HasSaveCheckpointsSecs, sparkdl.estimators.horovod_estimator.params.HasValidationInterval)
 Creates a new HorovodEstimator to use for distributed model training on a DataFrame.
 Args:
 modelFn (required):
 Defines the model and its outputs during training, evaluation, and inference. Run in
 training mode on each worker machine via MPI. Expected to have the same
 method signature as that expected by the tf.estimator.Estimator framework
 (See https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator#model_fn), with
 additional restrictions as described below.
 Arguments:
 * features: Dict of string `features_key` to tensor `T`, where
 (`input_col_name`, `features_key`) is a (key, value) pair in `featureMapping`
 and `T` is a tensor corresponding to batch of data from the DataFrame column
 with name `input_col_name`. For example, given a DataFrame containing an
 integer column "height_col", including the key-value pair
 {"height_col": "height_tensor"} would allow batches of data from "height_col"
 to be accessed via features["height_tensor"] in `modelFn`.
 * labels: Tensor, batch of labels
 * mode: Optional. Specifies if this training, evaluation or prediction. See
 tf.estimator.ModeKeys.
 * params: Optional dict of hyperparameters. Will receive what is passed to
 HorovodEstimator in params parameter. This allows for configuring Estimators for
 hyperparameter tuning.
 Returns: tf.estimator.EstimatorSpec. The export_outputs attribute (dict) of the EstimatorSpec
 must contain tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY
 as a key, with a corresponding value V of type tf.estimator.export.ExportOutput
 describing a set of model outputs. HorovodEstimator's fit() method returns a
 TFTransformer that produces the set of outputs V.
 Note that HorovodEstimator does not yet support passing the config argument to `model_fn`
 featureMapping (required): Dictionary from input column name to key in `features` dict passed
 to model_fn.
 labelCol (required): DataFrame column containing labels to feed to model_fn
 modelDir (required): Directory used for saving model checkpoints
 maxSteps (required): Number of training steps to run
 batchSize (required): Number of rows per batch of data fed to model_fn during model training
 and evaluation.
 isValidationCol (required): Name of boolean col indicating whether each row is a training
 or validation data point. We expect the column to be to True
 for each Row in the validation set, False for each row in the
 training set. WARNING: The specified validation dataset must
 fit on the driver machine's local disk.
 modelFnParams (optional): Additional `params` argument passed to model_fn. Can be used to
 pass hyperparameters for hyperparameter tuning, etc.
 saveCheckpointsSecs (optional): How often (in seconds) to save model checkpoints. Defaults
 to 600.
 validationInterval (optional): The frequency to do the validation for the saved model
 checkpoints. 0 means no validation. k(>0) means do validation
 after every k model checkpoints. Defaults to 1.
 
 Limitations of the current API:
 * Size restrictions on training & validation data:
 Training data for the estimator must fit across the local disks of machines in the cluster,
 and validation data for the estimator must fit on the driver's local disk.
 * Can only train o

Define the DL model to train in a Python function.

The tf.estimator-style `model_fn` ([see TensorFlow docs](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator)) works by:
1. Defining the model's network structure.
2. Specifying the model's output on a single batch of data during training, eval, and prediction (inference) phases.

**Note**: If you have a `model_fn` written for single-machine training, you can prepare it for distributed training with a one-line code change. Simply wrap your optimizer in a `HorovodDistributedOptimizer`, as in the example below.

In [0]:
def model_fn(features, labels, mode, params):
    """
    Arguments:
        * features: Dict of DataFrame input column name to tensor (each tensor corresponding to
                    batch of data from the input column)
        * labels: Tensor, batch of labels
        * mode: Specifies if the estimator is being run for training, evaluation or prediction.
        * params: Optional dict of hyperparameters. Will receive what is passed to
                  HorovodEstimator in params parameter. This allows for configuring Estimators for
                  hyperparameter tuning.
    Returns: tf.estimator.EstimatorSpec describing our model.  
    """
    from tensorflow.examples.tutorials.mnist import mnist
    # HorovodEstimator feeds scalar Spark SQL types to model_fn as tensors of shape [None]
    # (i.e. a variable-sized batch of scalars), and array Spark SQL types (including 
    # VectorUDT) as tensors of shape [None, None] (i.e. a variable-sized batch of dense variable-length arrays).
    #
    # Here image data is fed from an ArrayType(FloatType()) column,
    # e.g. as a float tensor with shape [None, None]. We know each float array is of length 784,
    # so we reshape our tensor into one of shape [None, 784].
    input_layer = features['image']
    input_layer = tf.reshape(input_layer, shape=[-1, 784])
    logits = mnist.inference(input_layer, hidden1_units=params["hidden1_units"],
                             hidden2_units=params["hidden2_units"])
    serving_key = tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY
    # Generate a dictionary of inference output name to tensor (for PREDICT mode)
    # Tensor outputs corresponding to the DEFAULT_SERVING_SIGNATURE_DEF_KEY are produced as output columns of
    # the TFTransformer generated by fitting our estimator
    predictions = {
        "classes": tf.argmax(input=logits, axis=1, name="classes_tensor"),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor"),
    }
    export_outputs = {serving_key: tf.estimator.export.PredictOutput(predictions)}
    # If the estimator is running in PREDICT mode, you can stop building our model graph here and simply return
    # our model's inference outputs
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions,
                                          export_outputs=export_outputs)
    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    if mode == tf.estimator.ModeKeys.TRAIN:
        # Set up logging hooks; these run on every worker.
        logging_hooks = [tf.train.LoggingTensorHook(tensors={"predictions": "classes_tensor"}, every_n_iter=5000)]
        # Horovod: scale learning rate by the number of workers, add distributed optimizer
        optimizer = tf.train.MomentumOptimizer(
            learning_rate=0.001 * hvd.size(), momentum=0.9)
        optimizer = hvd.DistributedOptimizer(optimizer)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op,
                                          export_outputs=export_outputs,
                                          training_hooks=logging_hooks)
    # If running in EVAL mode, add model evaluation metrics (accuracy) to your EstimatorSpec so that
    # they're logged when model evaluation runs
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(
        labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops, export_outputs=export_outputs)


Create a `HorovodEstimator` for distributed training of your model on a DataFrame.

In [0]:
# Model checkpoints will be saved to the driver machine's local filesystem.
model_dir = "/tmp/horovod_estimator"
dbutils.fs.rm(model_dir[5:], recurse=True)
# Create estimator
est = HorovodEstimator(modelFn=model_fn,
                       featureMapping={"image": "image"},
                       modelDir=model_dir,
                       labelCol="label_col",
                       batchSize=64,
                       maxSteps=5000,
                       isValidationCol="isVal",
                       modelFnParams={"hidden1_units": 100, "hidden2_units": 50},
                       saveCheckpointsSecs=30)

Fit the estimator on the MNIST dataframe, then use the fitted model (a `TFTransformer` from Deep Learning Pipelines) to transform your data.
Fitting the estimator produces the following in `modelDir`:
* Checkpoint files that can be used to restore the model to resume training
* Event files containing metrics logged during training & model evaluation, visualizable through TensorBoard
* When training finishes: a tf.SavedModel in `modelDir` that can be used for inference.

New "classes" and "probabilities" output columns are added to the DataFrame by the transformer.

**Note:** If using a GPU cluster, we recommend not opening any other TensorFlow sessions on the current cluster (in this notebook or in other notebooks on the same cluster).
If you open a TensorFlow session, the Python process running your notebook will use a GPU, preventing
HorovodEstimator from running. In this case you may need to detach/reattach your notebook, and rerun your HorovodEstimator code
without running any TensorFlow code beforehand.

In [0]:
# Add column indicating whether each row is in the training/validation set; we perform a random split of the data
df_with_val = df.withColumn("isVal", when(rand() > 0.8, True).otherwise(False))
# Fit estimator to obtain a TFTransformer
transformer = est.fit(df_with_val)
# Apply the TFTransformer to our training data and display the results. Note that our predicted "classes" tend to
# match the label column in our training set.
res = transformer.transform(df)
res.show()

WARNING:tensorflow:Hit error when running nvidia-smi: [Errno 2] No such file or directory: 'nvidia-smi': 'nvidia-smi'. Defaulting to running a single training process per machine. 
INFO:tensorflow:Running Horovod training command mpirun --allow-run-as-root -bind-to none -map-by slot -H 10.139.64.13:1,10.139.64.14:1,10.139.64.15:1 -output-filename /tmp/horovod_estimator -np 3 -x NCCL_DEBUG -x PROG_SPEC_PATH /databricks/python/bin/python /tmp/tmpnyseumrb/horovod_launcher.py
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
Warning: Permanently added '10.139.64.14' (ECDSA) to the list of known hosts.
Warning: Permanently added '10.139.64.13' (ECDSA) to the list of known hosts.
WARNING:tensorflow:Using temporary folder as model directory: /tmp/tmp4a5uhlsz
WARNING:tensorflow:Using temporary folder as model directory: /tmp/tmpy2yeu2km
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp4a5uhlsz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 30, '_session_config': gpu_options {
 allow_growth: true
 visible_device_list: "0"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f89b13426d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpy2yeu2km', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 30, '_session_config': gpu_options {
INFO:tensorflow:Using config: {'_model_dir': '/tmp/horovod_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 30, '_session_config': gpu_options {
 allow_growth: true
 visible_device_list: "0"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f15fc098860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running eval until 5000 training steps have passed 
INFO:tensorflow:Eval: Couldn't find checkpoint in directory /tmp/horovod_estimator, sleeping for 15.0 seconds...
 allow_growth: true
 visible_device_list: "0"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f64906f36d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using config: {'_model_dir': '/tmp/horovod_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 30, '_session_config': gpu_options {
 allow_growth: true
 visible_device_list: "0"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensor

As stated above, you can restore our model from a checkpoint file to resume training. Here increase the `maxSteps` parameter of your estimator & run training again, starting from your previously-saved checkpoint:

In [0]:
est.setMaxSteps(10000)
new_transformer = est.fit(df_with_val)
new_res = transformer.transform(df)
new_res.show()

WARNING:tensorflow:Hit error when running nvidia-smi: [Errno 2] No such file or directory: 'nvidia-smi': 'nvidia-smi'. Defaulting to running a single training process per machine. 
INFO:tensorflow:Running Horovod training command mpirun --allow-run-as-root -bind-to none -map-by slot -H 10.139.64.14:1,10.139.64.13:1,10.139.64.15:1 -output-filename /tmp/horovod_estimator -np 3 -x NCCL_DEBUG -x PROG_SPEC_PATH /databricks/python/bin/python /tmp/tmp8567ky33/horovod_launcher.py
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
WARNING:tensorflow:Using temporary folder as model directory: /tmp/tmps1h0xy41
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmps1h0xy41', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 30, '_session_config': gpu_options {
 allow_growth: true
 visible_device_list: "0"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5c6986d748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
WARNING:tensorflow:Using temporary folder as model directory: /tmp/tmpldyb6z3v
INFO:tensorflow:Using config: {'_model_dir': '/tmp/horovod_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 30, '_session_config': gpu_options {
 allow_growth: true
 visible_device_list: "0"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa723fdbd68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpldyb6z3v', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 30, '_session_config': gpu_options {
INFO:tensorflow:Using config: {'_model_dir': '/tmp/horovod_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 30, '_session_config': gpu_options {
 allow_growth: true
 visible_device_list: "0"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdbae406860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running eval until 10000 training steps have passed 
INFO:tensorflow:Detected new checkpoint 5000
INFO:tensorflow:Running eval
 allow_growth: true
 visible_device_list: "0"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9287743748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': Tr

## Copy model checkpoints to DBFS
Persist your model checkpoints & event files by copying them to DBFS, so that they can be accessed e.g. after the current cluster terminates.

In [0]:
dbutils.fs.cp("file:/tmp/horovod_estimator/", "dbfs:/horovod_estimator/", recurse=True)

Out[8]: True

You can copy this directory to your local machine using the [Databricks CLI](https://pypi.org/project/databricks-cli/).

```sh
$ databricks fs cp -r 'dbfs:/horovod_estimator' ./local/path/horovod_estimator
```